Loading of libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import scipy as sp
import matplotlib.pyplot as plt

import os
import sys
import operator
from scipy import sparse
import xgboost as xgb
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

Required for this round of exploration

Read files

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
train_df=pd.read_json('../input/train.json')
test_df=pd.read_json('../input/test.json')
print('Training data dimensions:',train_df.shape)
print('Testing data dimensions:',test_df.shape)

Set up for KNN longitude and lattitiude

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=39)
def LongLatLoc(train_df,model,mode) :
    X=pd.concat([train_df['latitude'],train_df['longitude']],axis=1)
    if mode==1:
        y=train_df['interest_level']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=45)
        model.fit(X_train, y_train)
        predVal=model.predict(X_test)
    else :
        predVal=model.predict(X)
        y_test='N/A'
    return predVal, model, y_test    

In [ ]:
def checkpred(predVal,y_test,mode):
    if mode==1:
        mat=[predVal,y_test]
    else :
        mat='N/A'
    df=pd.DataFrame(mat).transpose()
    df.columns=('h0','y')
    df['diff']=np.where(df.h0==df.y,1,0)
    print('% correct =',sum(df['diff'])/len(df['diff'])*100)

In [ ]:


def addnewcol(df,ary,fieldname):
	df['tem2349field']=ary
	target_num_map = {'high':0, 'medium':1, 'low':2}
	df[fieldname]=df['tem2349field'].apply(lambda x: target_num_map[x])
    df.drop('tem2349field', axis=1, inplace=True)
    return df

In [ ]:
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0, num_rounds=1000):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1
    param['max_depth'] = 6
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds
    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

check with splitting the input as test and train

In [ ]:
predVal,neigh,y_test=LongLatLoc(train_df,neigh,1)
checkpred(predVal,y_test)

Generate the field for all test data

In [ ]:
predVal,neigh,y_test=LongLatLoc(train_df,neigh,0)
y_test=train_df['interest_level']
checkpred(predVal,y_test)

Add the prediction to the features

In [ ]:
"""
trainDF['locvalst']=predVal
target_num_map = {'high':0, 'medium':1, 'low':2}
trainDF['locval']=trainDF['locvalst'].apply(lambda x: target_num_map[x])
"""
train_df=addnewcol(train_df,predVal,'locval')
#train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

In [ ]:
predVal,neigh,y_test=LongLatLoc(test_df,neigh,0)
test_df=addnewcol(test_df,predVal,'locval')
"""
testDF['locvalst']=predVal
testDF['locval']=testDF['locvalst'].apply(lambda x: target_num_map[x])
"""

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
features_to_use  = ["bathrooms", "bedrooms", "latitude", "longitude", "price", "locval"]

In [ ]:
# count of photos #
train_df["num_photos"] = train_df["photos"].apply(len)
test_df["num_photos"] = test_df["photos"].apply(len)

# count of "features" #
train_df["num_features"] = train_df["features"].apply(len)
test_df["num_features"] = test_df["features"].apply(len)

# count of words present in description column #
train_df["num_description_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))
test_df["num_description_words"] = test_df["description"].apply(lambda x: len(x.split(" ")))

# convert the created column to datetime object so as to extract more features 
train_df["created"] = pd.to_datetime(train_df["created"])
test_df["created"] = pd.to_datetime(test_df["created"])

# Let us extract some features like year, month, day, hour from date columns #
train_df["created_year"] = train_df["created"].dt.year
test_df["created_year"] = test_df["created"].dt.year
train_df["created_month"] = train_df["created"].dt.month
test_df["created_month"] = test_df["created"].dt.month
train_df["created_day"] = train_df["created"].dt.day
test_df["created_day"] = test_df["created"].dt.day
train_df["created_hour"] = train_df["created"].dt.hour
test_df["created_hour"] = test_df["created"].dt.hour

# adding all these new features to use list #
features_to_use.extend(["num_photos", "num_features", "num_description_words","created_year", "created_month", "created_day", "listing_id", "created_hour"])

In [ ]:
categorical = ["display_address", "manager_id", "building_id", "street_address"]
for f in categorical:
        if train_df[f].dtype=='object':
            #print(f)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_df[f].values) + list(test_df[f].values))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))
            features_to_use.append(f)

In [ ]:
train_df['features'] = train_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
test_df['features'] = test_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
print(train_df["features"].head())
tfidf = CountVectorizer(stop_words='english', max_features=200)
tr_sparse = tfidf.fit_transform(train_df["features"])
te_sparse = tfidf.transform(test_df["features"])

In [ ]:
train_X = sparse.hstack([train_df[features_to_use], tr_sparse]).tocsr()
test_X = sparse.hstack([test_df[features_to_use], te_sparse]).tocsr()

target_num_map = {'high':0, 'medium':1, 'low':2}
train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))
print(train_X.shape, test_X.shape)

In [ ]:
cv_scores = []
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
for dev_index, val_index in kf.split(range(train_X.shape[0])):
        dev_X, val_X = train_X[dev_index,:], train_X[val_index,:]
        dev_y, val_y = train_y[dev_index], train_y[val_index]
        preds, model = runXGB(dev_X, dev_y, val_X, val_y)
        cv_scores.append(log_loss(val_y, preds))
        print(cv_scores)
        break

In [ ]:
preds, model = runXGB(train_X, train_y, test_X, num_rounds=400)
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_csv("xgb_starter2.csv", index=False)